In [ ]:
import numpy as np
import rasterio
import geopandas as gpd
from rasterio.enums import Resampling
from rasterio.transform import from_origin, Affine
from rasterio.mask import mask
import geopandas as gpd
from scipy.interpolate import griddata
from osgeo import gdal, gdalconst
import xdem
import matplotlib.pyplot as plt

### 1. Loading and inspecting raster data with rasterio

To explore how raster data is stored in python when it is loaded with rasterio by answering the following questions:

1) What is stored in dem_src, what is stored in dem2020?
2) Explore the metadata. What are the dimensions, coordinate reference system, and resolution of this dataset?
3) Can you also derive the no-data value, and the elevation range of the DEM?
4) Plot the 2020 DEM (define some reasonable minimum and maximum values). What do you notice about the axes? Where in space is this data?
5) Plot the data again, passing plt.imshow() the correct extent.

In [ ]:
#load with rasterio
with rasterio.open('./data/DEMs/Silvretta-2020-2m.tif') as dem_src:
    dem2020 = dem_src.read(1)

In [ ]:
# What datatypes are dem_src and dem2020, and what does dem2020 contain?
type(...)

In [ ]:
# Explore metadata
# What is the file resolution, the image dimensions, the spatial resolution, and the no-data value?
dem_src.meta
 

In [ ]:
# What is the elevation range of the Digital Elevation Model?

In [ ]:
#mask no-data values for prettyness:
if dem_src.nodata is not None:
    dem2020 = np.ma.masked_equal(dem2020, dem_src.nodata)

In [ ]:
# Plot elevation using a reasonable range for the elevation. How might you query this if you don't know?
f, ax = plt.subplots(figsize=(8,8))
cax = ax.imshow(dem2020,
           vmin=..., 
           vmax=..., 
           cmap='terrain')
cbar = f.colorbar(cax, ax=ax, label='Elevation', shrink=0.5)
ax.set_title('Silvretta elevation')
plt.show()

In [ ]:
# Plot elevation so that the data ends up on the right place on Earth!

# Use the bounds and attributes thereof to define the spatial extent of the DEM. 
bounds = dem_src.bounds
extent = [..., ..., ..., ...] #left, right, bottom, top

In [ ]:
f, ax = plt.subplots(figsize=(8,8))
cax = ax.imshow(dem2020,
           extent=...,
           vmin=..., 
           vmax=..., 
           cmap='terrain')
cbar = f.colorbar(cax, ax=ax, label='Elevation', shrink=0.5)
ax.set_title('Silvretta elevation')
plt.show()

### 2. Loading, reprojecting, and manipulating raster data with xdem

xdem is designed to work with elevation data, but it can also be used for other raster data. It's built on rasterio (and by association gdal), but it's a bit more packaged. Load the same dataset again and

1) Use the .plot() method and see how it plots. What do you notice / what is different with regard to using rasterio?
2) Take a look a the xdem [terrain attributes](https://xdem.readthedocs.io/en/stable/terrain.html) and try to plot a hillshade.
3) Load the ice thickness estimate from [Farinotti et al., 2019](https://www.nature.com/articles/s41561-019-0300-3) for Silvretta glacier and plot it. What do you notice with this?
4) Create a plot that shows the ice thickness over the hillshade, including the glacier outline and a colorbar for the ice thickness!
5) Already done? Load a second DEM and create and plot a difference map, showing the glacier outlines as well.


In [ ]:
dem = xdem.DEM(...) #load the dataset

In [ ]:
dem.plot() #notice how it already knows where it is in the world

In [ ]:
# Plot a hillshade. You might want to define a more appropriate colormap. 
... .plot()

In [ ]:
# write the hillshade to file for future use!
hillshade = ...

Let's use xdem to load another raster dataset, namely the ice thickness estimate

In [ ]:
rgi_thickness = xdem.DEM('./data/RGI60-11/RGI60-11.00804_thickness.tif') #Ice thickness estimate for Silvretta glacier from a global consensus estimate

In [ ]:
# plot the ice thickness! Can you plot it with the DEM?

What is the problem? Take a look here for a solution (once you have identified the problem ;-) [hint](https://xdem.readthedocs.io/en/stable/gen_modules/xdem.DEM.reproject.html))

In [ ]:
# fix the problem! 
thickness_lv95 = ...

In [ ]:
# load our outline 
outline = ...

In [ ]:
#plot!
f, ax = plt.subplots()
hillshade.plot(ax=ax,
               cmap=...,
               add_cbar=...,
               alpha=...)

thickness_lv95.plot(ax=ax, 
              cmap=..., 
              cbar_title=..., 
              alpha=...)
outline.plot(...)
ax.set_xlim(..., ...)
ax.set_ylim(..., ...)
# add title
# add axes labels

In [ ]:
# loading a second DEM and differencing them

In [ ]:
dem2014 = ...

In [ ]:
dem2014_reproj = ...

In [ ]:
diff = ...

In [ ]:
sgi = ...

In [ ]:
f, ax = plt.subplots()
...
...
ax.set_xlim(..., ...)
ax.set_ylim(..., ...)

### 3. Extracting data from rasters: points, polygons, multi-polygons

Oftentimes, we want to get information from raster datasets that are linked to vector datasets, e.g., values at points or within on or more polygons. Use the following lines of code to 

1) Extract the minimum, maximum, mean, median, and standard deviation of the ice thickness inside our glacier outline using [rasterio.mask](https://rasterio.readthedocs.io/en/stable/api/rasterio.mask.html)
2) Calculate the total ice volume! What additional information do you need?
3) Extract the minimum, maximum, mean, median for the sampling points you defined earlier.
4) Finally, try to adapt the code to work for a set of polygons (buffers around your points)

The ice thickness dataset is currently an xdem.DEM, which is not (yet) as versatile as their rasterio counterparts. Therefore, xdem has a to_rio_dataset method, which easily allows you to convert to a rasterio dataset and use all the methods there. Therefore, our first step is to convert from xdem.dem.DEM to rasterio.io.DatasetReader

In [ ]:
thickness_rio = thickness_lv95.to_rio_dataset()

In [ ]:
# use rasterio.mask() to mask 
masked_data, masked_transform = mask(..., ..., crop=True, nodata=thickness_rio.nodata)

In [ ]:
# Set the no-data values to NaNs to exclude them from the analysis
masked_data = np.ma.masked_equal(masked_data, thickness_rio.nodata)

In [ ]:
stats = {
    'min': masked_data.min(),
    'max': ...,
    'mean': ...,
    'median': np.ma.median(masked_data),
    'std': ...
}

print("Zonal Statistics:")
for key, value in stats.items():
    print(f"{key}: {value}")

Calculate the volume of ice inside the outline! What additional information do you need?

In [ ]:
V_ice = ...

Now let's do this for our point dataset:

In [ ]:
# sampling a raster dataset at predefined points or extracting information inside a 
# load geojson
sampling_points = ...

In [ ]:
# Create a list to store raster values at points
raster_values_at_points = []

# Sample raster at each point
for index, point in sampling_points.iterrows():
    # Convert point coordinates to pixel coordinates
    pixel_coords = rasterio.transform.rowcol(thickness_rio.transform, point.geometry.x, point.geometry.y)

    # Get raster value at the pixel coordinate
    value = thickness_rio.read(1)[pixel_coords]

    # Add value to the list
    raster_values_at_points.append(value)

# Add raster values to GeoDataFrame
sampling_points['ice_thickness'] = raster_values_at_points

print(sampling_points)

Now we want to get stats for a series of polygons. Let's say we don't trust the individual point measurements, so we put a 50 m buffer around our initial points. 

In [ ]:
sampling_points['buffered'] = ...

In [ ]:
buffered_points = sampling_points.copy()
buffered_points = buffered_points.drop(['geometry','ice_thickness'], axis=1) #drop the point geometry and ice thickness attribute

In [ ]:
# Take a look at the new geodataframe.  Can you reconstruct what the past few commands did?
buffered_points

In [ ]:
buffered_points = buffered_points. ... # set the geometry to "buffered"

Use your knowledge of the single-polygon and multiple point examples to complete the code below:

In [ ]:
# Create a list to store raster values for polygons
raster_stats_for_polygons = ...

# iterate over the rows of the dataframe:
for index, row in sampling_points.iterrows():

    # Run rasterio.mask on the polygon of the current row in the dataframe. Check the documentation if you are unsure about what datatypes 
    # the arguments reguire. 
    masked_data, masked_transform = ...
    
    # Exclude nodata values from the masked array
    masked_data = ...
    
    # Calculate statistics of the masked array
    stats = {
            ...
    }
    
    # Add statistics to the list
    raster_stats_for_polygons.append(stats)

# Add raster statistics to GeoDataFrame
buffered_points = buffered_points.assign(**{key: [stat[key] for stat in raster_stats_for_polygons] for key in raster_stats_for_polygons[0].keys()})

print(buffered_points)

### 4. Writing raster data

Finally, let's look at how we can write out raster data that we create inside python as a geotiff (e.g. for sharing with others, or reloading for further processing)

1) Load the points from glathida that are inside the glacier outline.
2) Interpolate the ice thickness between the points.
3) Save the interpolated grid as a geotiff

Let's load some old data that we previously created

In [ ]:
points_inside = gpd.read_file('data/glathida_inside_sgi.geojson')

In [ ]:
# Extract the bounds of the polygon
minx, miny, maxx, maxy = outline.total_bounds
# Create an empty grid for interpolation
cell_size = 10 #m
grid_x, grid_y = np.meshgrid(np.arange(minx, maxx, cell_size), np.arange(maxy, miny, -cell_size)) # for a grid with 10m spacing

In [ ]:
points = np.column_stack([points_inside.geometry.x, points_inside.geometry.y])
values = points_inside['THICKNESS'].values

In [ ]:
grid_z = griddata(points, values, (grid_x, grid_y), method='linear') #try also cubic. Which one looks better?

In [ ]:
#check your work!
f, ax = plt.subplots()
#using imshow
ax.imshow(grid_z, extent=[grid_x.min(), grid_x.max(), grid_y.min(), grid_y.max()], cmap='Blues')
#using contour
#ax.contourf(grid_x, grid_y, grid_z, cmap='Blues')

#add points from measurements
d = plt.scatter(
    points_inside.geometry.x,  # x-coordinates of points
    points_inside.geometry.y,  # y-coordinates of points
    c=points_inside['THICKNESS'],  # values to use for coloring
    cmap='Blues',  # colormap
    s=75,  # size of markers
    edgecolor='none',  # edge color of markers
    alpha=1  # transparency
)
outline.plot(ax=ax, facecolor='None')

In [ ]:
# set the affine transform parameters
pixel_width = cell_size
pixel_height = cell_size
transform = Affine.translation(grid_x.min(), grid_y.max()) * Affine.scale(pixel_width, -pixel_height)

In [ ]:
# set the relevant metadata
metadata = {
    'driver': 'GTiff',
    'dtype': 'float32',
    'nodata': ..., # set appropriate no-data value
    'width': grid_z.shape[1],
    'height': grid_z.shape[0],
    'count': 1,
    'crs': ...,  # Example EPSG code, replace with appropriate CRS
    'transform': transform
}

In [ ]:
with rasterio.open('data/glathida_interpolated.tif', 'w', **metadata) as dst:
    dst.write(grid_z.astype('float32'), 1)
